In [1]:
import numpy as np
from transformers import BlipProcessor , BlipForConditionalGeneration
from PIL import Image
from fpdf import FPDF 
import os 
import google.generativeai as genai
from PIL import Image
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
from datetime import datetime
import geocoder
import random as rd
import uuid 
import markdown
from xhtml2pdf import pisa
from io import BytesIO

from dotenv import load_dotenv 
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-flash")

e:\College\Mind Cloud Team\ERC\erc-vision-25\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
#---------------------------------------------------------------------------------------
def latlon_lines_on_map(latitude, longitude, image_path, save_path="map_edit.jpg"):
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    width, height = img.width, img.height
    def latlon_to_pixel(lat, lon):
        x = (lon + 180) * (width / 360)
        y = (90 - lat) * (height / 180)
        return int(x), int(y)
    _, y = latlon_to_pixel(latitude, 0)
    draw.line([(0, y), (width, y)], fill="red", width=3)
    x, _ = latlon_to_pixel(0, longitude)
    draw.line([(x, 0), (x, height)], fill="red", width=3)
    img.save(save_path)
#---------------------------------------------------------------------------------------   
def save_image_grid(image_dir, max_images=15, rows=None, cols=None, figsize=(12, 8), save_path="comb.jpg"):
    valid_ext = [".jpg", ".jpeg", ".png", ".bmp", ".gif"]

    image_paths = [os.path.join(image_dir, fname)
                   for fname in os.listdir(image_dir)
                   if os.path.splitext(fname)[1].lower() in valid_ext]

    image_paths = image_paths[:max_images]
    num_images = len(image_paths)

    if rows is None or cols is None:
        cols = min(5, num_images)
        rows = math.ceil(num_images / cols)

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.ravel() if num_images > 1 else [axes]

    for i in range(rows * cols):
        ax = axes[i]
        if i < num_images:
            img = mpimg.imread(image_paths[i])
            ax.imshow(img)
        ax.axis("off")
    plt.tight_layout()

    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()
features_dir = os.path.join(os.getcwd(), "features")
image_paths = [os.path.join(features_dir, fname) for fname in os.listdir(features_dir) if fname.endswith((".jpg", ".png", ".jpeg"))]
#---------------------------------------------------------------------------------------
def clean_text(text):
    """Clean text for basic PDF compatibility"""
    import unicodedata
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("latin-1", errors="ignore").decode("latin-1")
    return text

def render_markdown_to_pdf(markdown_text, output_path):
    """Convert markdown text to PDF using HTML as intermediate format"""
    # Convert markdown to HTML
    html = markdown.markdown(markdown_text)
    
    # Add basic CSS styling for better PDF output
    html_with_css = f"""
    <html>
    <head>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 40px;
            }}
            h1, h2, h3 {{
                color: #333;
            }}
            strong {{
                font-weight: bold;
            }}
            em {{
                font-style: italic;
            }}
            img {{
                max-width: 100%;
                height: auto;
                display: block;
                margin-left: auto;
                margin-right: auto;
            }}
        </style>
    </head>
    <body>
        {html}
    </body>
    </html>
    """
    
    # Convert HTML to PDF
    with open(output_path, 'wb') as result_file:
        pisa_status = pisa.CreatePDF(html_with_css, dest=result_file)
    
    return not pisa_status.err

def process_ai_text_for_markdown(text):
    """Process AI-generated text to ensure markdown formatting is preserved"""
    # Don't clean markdown syntax - preserve it for rendering
    return text
#---------------------------------------------------------------------------------------

In [3]:
current_dir = os.getcwd()
if "features" in os.listdir() :
    print("Done")
    path_of_images = os.path.join(current_dir,"features")
    print(os.listdir(path_of_images))
    
else :
    print("no features dir")
print(current_dir)

Done
['im1.jpeg', 'im10.jpg', 'im11.jpg', 'im12.jpg', 'im14.jpg', 'im2.jpg', 'im3.jpg', 'im4.jpg', 'im7.jpg', 'im8.png', 'im9.jpg', 'temp_converted']
e:\College\Mind Cloud Team\ERC\erc-vision-25\PDF


In [4]:
# Updated clean_text function that preserves markdown
import unicodedata

def clean_text_preserve_markdown(text):
    """Preserve markdown formatting while normalizing Unicode"""
    # Normalize Unicode but don't strip markdown syntax
    text = unicodedata.normalize("NFKD", text)
    # Only remove problematic characters, keep markdown syntax
    return text

In [13]:
# Modified PDF generation with markdown support
markdown_content = []

# Add title and introduction
markdown_content.append("# Report of Mission - Mind Cloud Team")
markdown_content.append("## A. Introduction")

# Read introduction text
try:
    with open("tet.txt", "r+") as file:
        intro_text = file.read()
        markdown_content.append(intro_text)
except FileNotFoundError:
    markdown_content.append("Introduction text file not found.")

markdown_content.append("\n---\n")
markdown_content.append("## B. Feature Analysis")

# Process images and generate analysis
image_folder = "../landmarks"
try:
    image_paths = [fname for fname in os.listdir(image_folder) if fname.endswith((".jpg", ".png", ".jpeg"))]
    
    if not image_paths:
        markdown_content.append("No images found in the specified folder.")
    else:
        for filename in image_paths[:5]:  # Limit to first 5 images for demo
            image_path = os.path.join(image_folder, filename)
            try:
                # Generate feature ID
                name = "F-ERC-" + str(rd.randint(10000, 99999)) + "K-M" + str(rd.randint(100, 999))
                
                markdown_content.append(f"### **Feature ID: {name}**")
                
                # Add image directly as an HTML img tag within markdown
                # Use a relative path that xhtml2pdf can resolve if run from the correct directory.
                # A more robust solution for production would involve embedding images or serving them via a temporary web server.
                markdown_content.append(f"<img src=\"{image_path}\" alt=\"{filename}\" style=\"width:100%;\">")
                
                # Generate analysis using AI
                try:
                    img_analysis = model.generate_content([
                        Image.open(image_path),
                        """Describe what you see in the marsyard for any notable features/objects. 
                        Format your response using markdown with **bold** for important terms and *italics* for emphasis.
                        Don't describe the image in perspective, just focus on the features.
                        Any landmarks found in marsyard should be mentioned.
                        Use bullet points where appropriate. Keep it to 1-2 sentences each maximum."""
                    ])
                    
                    findings = model.generate_content([
                        Image.open(image_path),
                        """Say the name of the findings in the image,
                        Output the findings in a list format
                        example: ['blue ball', 'robot', 'water bottle']"""
                    ])
                    
                    # Process the AI response to preserve markdown
                    analysis_text = process_ai_text_for_markdown(img_analysis.text)
                    markdown_content.append(f"**Analysis:**")
                    markdown_content.append(analysis_text)
                    
                except Exception as e:
                    markdown_content.append(f"**Analysis:** Error analyzing image: {str(e)}")
                
                markdown_content.append("\n---\n")
                
            except Exception as e:
                markdown_content.append(f"Error processing {filename}: {str(e)}")
                
except FileNotFoundError:
    markdown_content.append("Image folder not found.")

# Add summary section
markdown_content.append("## C. Summary of Mission")

try:
    # Generate summary with markdown formatting
    summary_prompt = f"""
    The findings from the mission have been compiled: {findings.text if len(findings.text.split(",")) > 0 else 'No analysis available.'}
    Generate a summary with the following structure using markdown formatting:
    
    **Key Findings:**
    - Use bullet points for main discoveries
    - Use **bold** for important terms
    - Use *italics* for emphasis
    
    Go into the details of each finding, explaining its significance and implications and its semantic relation to the environment.
    Keep the entire summary to about 100-200 words.
    """
    
    summary_response = model.generate_content(summary_prompt)
    summary_text = process_ai_text_for_markdown(summary_response.text)
    markdown_content.append(summary_text)
    
except Exception as e:
    markdown_content.append(f"Error generating summary: {str(e)}")

# Combine all markdown content
full_markdown = "\n\n".join(markdown_content)

# Generate PDF from markdown
output_pdf_path = "mission_report_with_markdown.pdf"
success = render_markdown_to_pdf(full_markdown, output_pdf_path)

if success:
    print(f"PDF with markdown formatting generated successfully: {output_pdf_path}")
else:
    print("Error generating PDF")
    
# Also save the markdown content for reference
with open("mission_report.md", "w", encoding="utf-8") as f:
    f.write(full_markdown)
    
print("Markdown file saved: mission_report.md")

PDF with markdown formatting generated successfully: mission_report_with_markdown.pdf
Markdown file saved: mission_report.md
